<a href="https://colab.research.google.com/github/Jeeeeeeeee/Study/blob/main/_%5BPytorch_01%5D_CIFAR10_%EB%AA%A8%EB%8D%B8_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms # 전처리
from torch.utils.data import DataLoader # Batch형태로 만들어주는 DataLoader

# Load data set

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5,0.5))]
)

trainset = torchvision.datasets.CIFAR10(root = './data', train = True,
                                        download = True, transform = transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size = 8,
                                          shuffle = True, num_workers = 2) # 얘 길이는 길어질 수 있다

testset = torchvision.datasets.CIFAR10(root = './data', train = False,
                                       download = True, transform = transform)

testloader = torch.utils.data.DataLoader(testset, batch_size = 8,
                                         shuffle = False, num_workers = 2)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', ' ship', ' truck')

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


# Build a model

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class Net(nn.Module): # nn.Module을 상속받기
  def __init__(self): ### 내가 사용하고자 하는 연산들을 정의해주는 부분

    super(Net, self).__init__() # 여기까진 fix
    # class니까, self 형태로 넣는다.

    self.conv1 = nn.Conv2d(3, 6, 5) # convolution layer
    self.pool = nn.MaxPool2d(2, 2) # MaxPooling layer
    self.conv2 = nn.Conv2d(6, 16, 5) 
    self.fc1 = nn.Linear(16*5*5, 120) # fully connected layer 3개를 넣어서 class가 10개기 때문에
    # output이 10개인 one-hot-vertor 형태로 바꿔줄 수 있는 출력이 되도록 만듬.
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)


  def forward(self, x): # 정의된 연산 쓸 때 self.dot (여기서는 사용)
  # X = input 
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = x.view(-1, 16*5*5) # size를 변형할 때 사용한다. fully connect에 넣어주기 위해서 flatten
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x # 연산의 순서를 정의해 주는 부분


net = Net() # class에 대한 인스턴스 -> net을 가지고 모델을 돌릴 수 있다.

In [ ]:
print(net)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


## Implement the model with training set

In [ ]:
import torch.optim as optim # Loss function과 Gradient를 구할 수 있는 optimizer 정의해준다.

criterion = nn.CrossEntropyLoss() # 분류 문제에 쓰이는 함수
optimizer = optim.SGD(net.parameters(), lr = 0.001, momentum = 0.9)

# 지도학습에서는 아웃풋이 나오고, 아웃풋을 가지고 loss를 구하고 loss룰 바탕으로 Gradient 값을 구하고
# 다시 모델에 있는 parameter를 업데이터 시켜주는 형태...

In [ ]:
for epoch in range(1): # 여기선 에폭 1인데 당연히 많아져야..
# 데이터를 많이 보게 하기 위해 epoch 설정.. 아니면 한번만 본다.

  running_loss = 0.0
  for i, data in enumerate(trainloader, 0):
    inputs, labels = data

    optimizer.zero_grad() # optimizer 초기화

    outputs = net(inputs) # 이미지를 net에 넣어준다.
    loss = criterion(outputs, labels) # output과 label 비교
    # -> CrossEntropy값 기준으로 loss 뽑아내고
    loss.backward() # loss 기준으로 Gradient 계산, backward 한다.
    optimizer.step() # optimizer 계산한다.

    # print datasets
    running_loss += loss.item() # 이거는 loss 를 계속 누적시켜 주기 위해서
    if i % 2000 == 1999:
      print('[%d, %5d] loss : %.3f' %
            (epoch + 1, i+1, running_loss / 2000))
      running_loss = 0.0

### batch형태로 계산하고, 8개로 되어있기 때문에 8개의 이미지에 대해 한번 다 하는 것.

print('Finished Training')

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


[1,  2000] loss : 2.227
[1,  4000] loss : 1.847
[1,  6000] loss : 1.612
Finished Training


# Save the trained model

- 무조건 epoch가 제일 높을 때 하는 게 아니라, 조건문을 줘서 best_model을 추려낼 수 있다.

In [ ]:
PATH = './cfar_net.pth'
torch.save(net.state_dict(), PATH)

# Load the trained model

In [ ]:
net = Net()
net.load_state_dict(torch.load(PATH))

<All keys matched successfully>

# Test

In [ ]:
correct = 0
total = 0

with torch.no_grad(): # test할 때는 Graident 추적, 업데이트 X
  for data in testloader: # testloader 배치로 나눴기 때문에
    images, labels = data
    outputs = net(images) # output에는 10개의 값에 대한 벡터가 나올 것.
    _, predicted = torch.max(outputs.data, 1) # 그중에 가장 큰거 뽑아서 예측하겠다. arguemax와 같다.
    # 가장 큰 자릿값을 가지고 오겠다,,,, 10개짜리가 하나가 될것.

    total += labels.size(0) #label도 하나이다.
    correct += (predicted == labels).sum().item() # label과 predicted가 맞으면 correct


print('Accuracy of the network on the 10000 test images : %d %%' % (100 * correct / total))

Accuracy of the network on the 10000 test images : 44 %


NameError: ignored

In [ ]:
ds2 = MyDataset(train_images, train_labels, transform = Mytransform())
train_loader2 = DataLoader(ds2, batch_siz?e = 10, shuffle = True)

SyntaxError: ignored

In [ ]:
first_data = ds2[0]
features, labels = first_data
print(type(features), type(labels))

NameError: ignored

In [ ]:
dataiter2 = iter(train_loader2)
images2, labels2 = dataiter2.next()

In [ ]:
images2.size()

- reference : Youtube 딥러닝 호형 님 강의